In [4]:
from azureml.core import Workspace, Experiment

#ws = Workspace.get(name="udacity-project")
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name=ws.name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-160168
Azure region: southcentralus
Subscription id: a0a76bad-11a1-4a2d-9887-97a29122c8ed
Resource group: aml-quickstarts-160168


In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

pipeline_cluster_name = "pipeline-cluster"

# Verify that the cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=pipeline_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, pipeline_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

InProgress....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [59]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({"--C": uniform(0.1, 1),
    "--max_iter": choice(100, 500, 1000)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")



In [64]:
# Create a SKLearn estimator for use with train.py

est = SKLearn("./",
compute_target="pipeline-cluster",
entry_script="train.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name='Accuracy',
    primary_metric_goal= PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
    max_concurrent_runs=2)

In [65]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

In [67]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print("best run details: ", best_run.get_details())
print("best run metrics :", best_run.get_metrics())

os.makedirs('./model', exist_ok=True)

for f in best_run.get_file_names():
    if f.startswith('outputs/model'):
        output_file_path = os.path.join('./model', 'hyperdrive_model.pkl')
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        best_run.download_file(name=f, output_file_path=output_file_path)

AttributeError: 'NoneType' object has no attribute 'get_details'

In [41]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path,
                                                validate=True,
                                                infer_column_types=True, 
                                                separator=',', 
                                                header=True, 
                                                support_multi_line=False,
                                                empty_as_string=False,
                                                encoding='utf8')

In [42]:
from train import clean_data
# Use the clean_data function to clean your data.
x, y = clean_data(ds)

data = x.join(y)

TypeError: 'NoneType' object is not iterable

In [47]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=data,
    label_column_name="y",
    n_cross_validations=5)

In [48]:
# Submit your automl run

automl_run = exp.submit(automl_config)

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_66d25001-7f55-4efe-b97c-6ac4d060eab3,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


In [54]:
# Retrieve and save your best automl model.

best_automl_run = automl_run.get_best_child()
print("best run details:", best_automl_run.get_details())
print("best run metrics :", best_automl_run.get_metrics())

for f in best_automl_run.get_file_names():
    if f.startswith('outputs/model'):
        output_file_path = os.path.join('./model', '/automl_model.pkl')
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        best_automl_run.download_file(name=f, output_file_path=output_file_path)

best run details: {'runId': 'AutoML_66d25001-7f55-4efe-b97c-6ac4d060eab3_30', 'status': 'Completed', 'startTimeUtc': '2021-10-06T15:41:14.513112Z', 'endTimeUtc': '2021-10-06T15:42:00.033788Z', 'services': {}, 'properties': {'runTemplate': 'automl_child', 'pipeline_id': '__AutoML_Ensemble__', 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'udacity-project\',\'compute_target\':\'local\',\'subscription_id\':\'a0a76bad-11a1-4a2d-9887-97a29122c8ed\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_66d25001-7f55-4efe-b97c-6ac4d060eab3_30","experiment_name":null,"workspace_name":"quick-starts-ws-160168","subscription_id":"a0a76bad-11a1-4a2d-9887-97a29122c8ed","res

In [ ]:
# clean up the cluster after use to save ressources
cpu_cluster.delete()